In [5]:
import yaml
import pyodbc
from market_growth_analysis.etl.stagging import *
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
# Load the YAML file
with open('../../conf/global.yml', 'r') as f:
    columns = yaml.safe_load(f)

# Load the YAML file
with open('../../conf/local.yml', 'r') as f:
    credentials = yaml.safe_load(f)

In [6]:
ratios_df = pd.read_csv("../../data/load_04/ratios_df.csv", index_col=0)
income_statememnt_df = pd.read_csv("../../data/load_04/income_statememnt_df.csv", index_col=0)
balance_sheet_df = pd.read_csv("../../data/load_04/balance_sheet_df.csv", index_col=0)
cash_flow_statement_df = pd.read_csv("../../data/load_04/cash_flow_statement_df.csv", index_col=0)
prices_df = pd.read_csv("../../data/load_04/prices_df.csv", index_col=0)
dim_grouped_df = pd.read_csv("../../data/load_04/dim_grouped_df.csv", index_col=0)

In [7]:
ratios_df['Date'] = pd.to_datetime(ratios_df['Date'], format='%Y-%m-%d')
income_statememnt_df['Date'] = pd.to_datetime(income_statememnt_df['Date'], format='%Y-%m-%d')
balance_sheet_df['Date'] = pd.to_datetime(balance_sheet_df['Date'], format='%Y-%m-%d')
cash_flow_statement_df['Date'] = pd.to_datetime(cash_flow_statement_df['Date'], format='%Y-%m-%d')
prices_df['Date'] = pd.to_datetime(prices_df['Date'], format='%Y-%m-%d')


In [8]:
financial_dict = {
'FACT_RATIOS': ratios_df,
'FACT_INCOME_STATEMENT': income_statememnt_df,
'FACT_BALANCE_SHEET': balance_sheet_df,
'FACT_CASH_FLOW_STATEMENT': cash_flow_statement_df,
'FACT_PRICES': prices_df,
'DIM_COMPANY': dim_grouped_df
}

In [9]:
driver = credentials['warehouse_db']['driver']
server = credentials['warehouse_db']['server']
database = credentials['warehouse_db']['database']
trusted = credentials['warehouse_db']['trusted_connection']
user = credentials['warehouse_db']['user']
password = credentials['warehouse_db']['password']


In [10]:
# Connection to input data

# Create the connection string
if user and password:
    connection_string = f'mssql+pyodbc://{user}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
else:
    connection_string = f'mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'

    
# Create the SQLAlchemy engine
engine = create_engine(connection_string)

In [11]:
# connection to create database

conn = pyodbc.connect('Driver={SQL Server};'
                      f'Server={server};'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [ ]:
# # Write the data to the SQL table
# ratios_df.to_sql(table_name, con=engine, if_exists='replace', index=False)

In [ ]:
def create_table_from_dataframe(df, table_name, db_name, cursor):
    # Drop the table if it exists
    drop_table_query = f'DROP TABLE IF EXISTS {db_name}.[dbo].[{table_name}];'
    cursor.execute(drop_table_query)

    # Generate the column definitions with matching data types
    columns = []
    for column_name, column_type in df.dtypes.items():
        if column_name == 'PK':
            columns.append(f'[{column_name}] [int] PRIMARY KEY')
        elif column_type == 'object':
            columns.append(f'[{column_name}] [varchar](MAX) NULL')
        elif column_type == 'int64':
            columns.append(f'[{column_name}] [int] NULL')
        elif column_type == 'float64':
            columns.append(f'[{column_name}] [float] NULL')
        elif column_type == 'datetime64[ns]':
            columns.append(f'[{column_name}] [datetime] NULL')
        elif column_type == 'bool':
            columns.append(f'[{column_name}] [bit] NULL')

    # Create the table query
    create_table_query = f'CREATE TABLE {db_name}.[dbo].[{table_name}] (\n'
    create_table_query += ',\n'.join(columns)
    create_table_query += '\n) ON [PRIMARY]'

    # Execute the CREATE TABLE statement
    cursor.execute(create_table_query)
    conn.commit() 



def insert_data_into_table(df, table_name, engine):
    # Write the data to the SQL table
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)


In [ ]:
for table in financial_dict.keys():
    create_table_from_dataframe(financial_dict[table], table, database, cursor)


In [ ]:
for table in financial_dict.keys():
    insert_data_into_table(financial_dict[table], table, engine)
